## IMPORTS and UTILS

In [ ]:
print("hi")

import os, math
import pandas as pd
import subprocess
import re
import numpy as np
import json
import csv

from joblib import Parallel, delayed



import matplotlib.pyplot as plt
import seaborn as sns
import random


import os

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC
from robust_evaluation_tools.robust_utils import get_site, robust_text, rwp_text, get_camcan_file, get_diseases, get_metrics, add_nb_patients_and_diseased, remove_covariates_effects_metrics
from robust_evaluation_tools.robust_harmonization import fit, apply, visualize_harmonization, QC, compare_with_compilation, compare_with_compilation_SMAPE, compare_with_compilation_STD, create_presentation, compare_distances, compare_with_compilation_var
from robust_evaluation_tools.synthectic_sites_generations import generate_sites
from robust_evaluation_tools.robust_outlier_detection import z_score_detection, flag_sid
from robust_evaluation_tools.robust_MLP import predict_malades_MLP

MAINFOLDER = "RESULTS/MAE_TEST"
SYNTHETIC_SITES = f"{MAINFOLDER}/SYNTHETIC_SITES"

ANALYSIS_FOLDER = f"{MAINFOLDER}/ANALYSIS"

robust_methods_for_analysis = ["No","raw", "IQR",'MAD','MMS', 'VS', 'VS2', 'FLIP', 'G_ZS', "G_ZS_IQR", "G_ZS_MAD"]

SMAPE_ONLY = False

## HARMONIZATION

In [ ]:
def harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, directory, method, robust, rwp,hc, gt_train_file_name, gt_test_file_name):
     
    if method == 'robust':
        dir =os.path.join(directory,robust)
    else:
        dir = os.path.join(directory,method)
    print(f_train)
    
    if robust == 'raw':
        output_filename_train = f_train
        output_filename_test = f_test
    else:
        # Fit the model
        output_model_filename = fit(f_train, ref_data_file, metric, harmonizartion_method, robust, rwp, dir, hc,)
        # Apply the model
        output_filename_train = apply(f_train, output_model_filename, metric, harmonizartion_method, robust, rwp, dir)
        output_filename_test = apply(f_test, output_model_filename, metric, harmonizartion_method, robust, rwp, dir)
    
    # Visualize the harmonization
    #visualize_harmonization(f_test, output_filename, ref_data_file, dir, bundles = '')
    
    std_mae_train = compare_with_compilation_STD(pd.read_csv(output_filename_train), pd.read_csv(gt_train_file_name))
    std_mae_test = compare_with_compilation_STD(pd.read_csv(output_filename_test), pd.read_csv(gt_test_file_name))

    if not SMAPE_ONLY:
        smape_test = compare_with_compilation_SMAPE(pd.read_csv(output_filename_test), pd.read_csv(gt_test_file_name))
        smape_train = compare_with_compilation_SMAPE(pd.read_csv(output_filename_train), pd.read_csv(gt_train_file_name))
        
        mae_test = compare_with_compilation(pd.read_csv(output_filename_test), pd.read_csv(gt_test_file_name))
        maev_test = compare_with_compilation_var(pd.read_csv(output_filename_test), pd.read_csv(gt_test_file_name))

        mae_train = compare_with_compilation(pd.read_csv(output_filename_train), pd.read_csv(gt_train_file_name))
        maev_train = compare_with_compilation_var(pd.read_csv(output_filename_train), pd.read_csv(gt_train_file_name))
    else:
        mae_test = pd.DataFrame()
        maev_test = pd.DataFrame()
        mae_train = pd.DataFrame()
        maev_train = pd.DataFrame()
        smape_test = pd.DataFrame()
        smape_train = pd.DataFrame()


    mae_test['site'] = get_site(f_train)
    mae_test['method'] = method
    mae_test['robust_method'] = robust

    maev_test['site'] = get_site(f_train)
    maev_test['method'] = method
    maev_test['robust_method'] = robust

    mae_train['site'] = get_site(f_train)
    mae_train['method'] = method
    mae_train['robust_method'] = robust

    maev_train['site'] = get_site(f_train)
    maev_train['method'] = method
    maev_train['robust_method'] = robust

    smape_test['site'] = get_site(f_train)
    smape_test['method'] = method
    smape_test['robust_method'] = robust

    smape_train['site'] = get_site(f_train)
    smape_train['method'] = method
    smape_train['robust_method'] = robust

    std_mae_train['site'] = get_site(f_train)
    std_mae_train['method'] = method
    std_mae_train['robust_method'] = robust

    std_mae_test['site'] = get_site(f_train)
    std_mae_test['method'] = method
    std_mae_test['robust_method'] = robust
    
    return mae_test, maev_test, mae_train, maev_train, smape_test, smape_train, std_mae_test, std_mae_train

In [ ]:
def analyse_site(f_train, f_test, robust_methods, directory, ref_data_file,
                 metric, harmonizartion_method, gt_train_file_name, gt_test_file_name):
    # Accumulateurs
    df_mae_test_robust   = pd.DataFrame()
    df_maev_test_robust  = pd.DataFrame()
    df_mae_train_robust  = pd.DataFrame()
    df_maev_train_robust = pd.DataFrame()
    df_smape_test_robust = pd.DataFrame()
    df_smape_train_robust = pd.DataFrame()
    df_std_mae_test_robust = pd.DataFrame()
    df_std_mae_train_robust = pd.DataFrame()

    for robust in robust_methods:
        if robust == "raw":
            (mae_test, maev_test, mae_train, maev_train,
             smape_test, smape_train, std_mae_test, std_mae_train) = harmonize(
                 f_train, ref_data_file, metric, harmonizartion_method,
                 f_test, directory, "raw", "raw", False, False, gt_train_file_name, gt_test_file_name)

        elif robust == "No":
            # hc‑only
            (mae_test_hc, maev_test_hc, mae_train_hc, maev_train_hc,
             smape_test_hc, smape_train_hc, std_mae_test_hc, std_mae_train_hc) = harmonize(
                 f_train, ref_data_file, metric, harmonizartion_method,
                 f_test, directory, "hc", "No", False, True, gt_train_file_name, gt_test_file_name)
            # NoRobust
            (mae_test_no, maev_test_no, mae_train_no, maev_train_no,
             smape_test_no, smape_train_no, std_mae_test_no, std_mae_train_no) = harmonize(
                 f_train, ref_data_file, metric, harmonizartion_method,
                 f_test, directory, "NoRobust", "No", False, False, gt_train_file_name, gt_test_file_name)

            mae_test   = pd.concat([mae_test_no,   mae_test_hc],   ignore_index=True)
            maev_test  = pd.concat([maev_test_no,  maev_test_hc],  ignore_index=True)
            mae_train  = pd.concat([mae_train_no,  mae_train_hc],  ignore_index=True)
            maev_train = pd.concat([maev_train_no, maev_train_hc], ignore_index=True)
            smape_test = pd.concat([smape_test_no, smape_test_hc], ignore_index=True)
            smape_train = pd.concat([smape_train_no, smape_train_hc], ignore_index=True)
            std_mae_test = pd.concat([std_mae_test_no, std_mae_test_hc], ignore_index=True)
            std_mae_train = pd.concat([std_mae_train_no, std_mae_train_hc], ignore_index=True)

        else:
            (mae_test, maev_test, mae_train, maev_train,
             smape_test, smape_train, std_mae_test, std_mae_train) = harmonize(
                 f_train, ref_data_file, metric, harmonizartion_method,
                 f_test, directory, "robust", robust, False, False, gt_train_file_name, gt_test_file_name)

        # Empile tout
        df_mae_test_robust   = pd.concat([df_mae_test_robust,   mae_test],   ignore_index=True)
        df_maev_test_robust  = pd.concat([df_maev_test_robust,  maev_test],  ignore_index=True)
        df_mae_train_robust  = pd.concat([df_mae_train_robust,  mae_train],  ignore_index=True)
        df_maev_train_robust = pd.concat([df_maev_train_robust, maev_train], ignore_index=True)
        df_smape_test_robust = pd.concat([df_smape_test_robust, smape_test], ignore_index=True)
        df_smape_train_robust = pd.concat([df_smape_train_robust, smape_train], ignore_index=True)
        df_std_mae_test_robust = pd.concat([df_std_mae_test_robust, std_mae_test], ignore_index=True)
        df_std_mae_train_robust = pd.concat([df_std_mae_train_robust, std_mae_train], ignore_index=True)

    return (df_mae_test_robust, df_maev_test_robust,
            df_mae_train_robust, df_maev_train_robust,
            df_smape_test_robust, df_smape_train_robust,
            df_std_mae_test_robust, df_std_mae_train_robust)

In [ ]:
def process_analysis(disease, sample_size, disease_ratio, test_index,
                     harmonization_method, SYNTHETIC_SITES_VERSION,
                     metrics, robust_methods):
    
    # Définition des chemins
    sizeDir = os.path.join(MAINFOLDER, f'PROCESS_{harmonization_method}', disease,
                           f"{sample_size}_{int(disease_ratio * 100)}",
                           f"{test_index}")
    mae_test_file_path  = os.path.join(sizeDir, "mae_compilation_test.csv")
    maev_test_file_path = os.path.join(sizeDir, "maev_compilation_test.csv")
    mae_train_file_path = os.path.join(sizeDir, "mae_compilation_train.csv")
    maev_train_file_path= os.path.join(sizeDir, "maev_compilation_train.csv")
    smape_test_file_path = os.path.join(sizeDir, "smape_compilation_test.csv")
    smape_train_file_path= os.path.join(sizeDir, "smape_compilation_train.csv")
    std_mae_train_file_path = os.path.join(sizeDir, "std_mae_compilation_train.csv")
    std_mae_test_file_path = os.path.join(sizeDir, "std_mae_compilation_test.csv")

    # Chargement existant (ou vide)
    def _load_if_exists(path):
        return pd.read_csv(path) if os.path.isfile(path) else pd.DataFrame()

    mae_compilation_test  = _load_if_exists(mae_test_file_path)
    maev_compilation_test = _load_if_exists(maev_test_file_path)
    mae_compilation_train = _load_if_exists(mae_train_file_path)
    maev_compilation_train= _load_if_exists(maev_train_file_path)
    smape_compilation_test = _load_if_exists(smape_test_file_path)
    smape_compilation_train = _load_if_exists(smape_train_file_path)
    std_mae_compilation_train = _load_if_exists(std_mae_train_file_path)
    std_mae_compilation_test = _load_if_exists(std_mae_test_file_path)


    robust_methods = ["No","raw"] + robust_methods
    # 💡 Vérification une seule fois au début : est-ce que certaines méthodes sont déjà présentes ?
    if not std_mae_compilation_test.empty:
        existing_methods = set(std_mae_compilation_test['robust_method'].unique())
        missing_methods = [m for m in robust_methods if m not in existing_methods]

        if not missing_methods:
            print(f"✔️ Toutes les méthodes déjà traitées pour {disease} {sample_size}_{int(disease_ratio*100)} test_index {test_index}.")
            return (mae_test_file_path, maev_test_file_path,
                    mae_train_file_path, maev_train_file_path,
                    smape_test_file_path, smape_train_file_path,
                    std_mae_test_file_path, std_mae_train_file_path
                    )
        
        # ⬇️ on ne garde que les méthodes manquantes
        robust_methods = missing_methods

    # Si on est ici, c’est qu’il reste des méthodes à faire
    directory_site = os.path.join(SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION, disease)
    sizeDir_site   = os.path.join(directory_site, f"{sample_size}_{int(disease_ratio * 100)}")
    tempDir_site   = os.path.join(sizeDir_site, f"{test_index}")

    df_for_tags = pd.read_csv(os.path.join(tempDir_site, f"train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_all.csv"))
    df_for_tags = df_for_tags[~df_for_tags['bundle'].isin(['left_ventricle', 'right_ventricle'])]
    df_for_tags = remove_covariates_effects_metrics(df_for_tags)

    z_score_sids = z_score_detection(df_for_tags)
    
    
        # — 1) Prépare les prédictions une seule fois —
    thresholds = [0.5, 0.6, 0.9, 0.95, 0.99]
    models     = ["mlp2_ALL", "mlp3_ALL", "mlp4_ALL"]          # ajoute-en d’autres au besoin

    preds = {f"{m.upper()}_{str(int(t*100)).rstrip('0')}": predict_malades_MLP(df_for_tags, m, threshold=t)
         for m in models for t in thresholds}



    for metric in metrics:
        tempDir = os.path.join(sizeDir, metric)
        os.makedirs(tempDir, exist_ok=True)

        # Préparation fichiers train/test
        train_file_name = f"train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"
        test_file_name  = f"test_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"
        gt_train_file_name = os.path.join(tempDir_site,f"gt_train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv")
        gt_test_file_name  = os.path.join(tempDir_site,f"gt_test_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv")

        train_file = os.path.join(tempDir_site, train_file_name)
        train_df = pd.read_csv(train_file)
        train_df = train_df[~train_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        train_df = train_df.drop(columns=['mean_no_cov', 'metric_bundle'])
        train_df = flag_sid(train_df, z_score_sids, "G_ZS")
        for label, sid_list in preds.items():
            train_df = flag_sid(train_df, sid_list, label)


        train_df["site"] = disease + "_" + train_df["site"]
        new_train_file = os.path.join(tempDir, train_file_name)
        train_df.to_csv(new_train_file, index=False)

        test_file = os.path.join(tempDir_site, test_file_name)
        test_df = pd.read_csv(test_file)
        test_df = test_df[~test_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        test_df = test_df.drop(columns=['mean_no_cov', 'metric_bundle'])
        test_df["site"] = train_df["site"]
        new_test_file = os.path.join(tempDir, test_file_name)
        test_df.to_csv(new_test_file, index=False)

        ref_data_file = get_camcan_file(metric)

        # Lancement de l’analyse pour toutes les méthodes manquantes
        mae_analyze_test, maev_analyze_test, mae_analyze_train, maev_analyze_train, \
        smape_analyze_test, smape_analyze_train, std_mae_analyze_test, std_mae_analyze_train = analyse_site(
            new_train_file, new_test_file, robust_methods, tempDir,
            ref_data_file, metric, harmonization_method, gt_train_file_name, gt_test_file_name
        )

        # Ajout des infos
        for df in [mae_analyze_test, maev_analyze_test, mae_analyze_train, maev_analyze_train, smape_analyze_test, smape_analyze_train, std_mae_analyze_test, std_mae_analyze_train]:
            df['disease'] = disease
            df['metric']  = metric

        std_mae_compilation_test = pd.concat([std_mae_compilation_test, std_mae_analyze_test],
                                                ignore_index=True).drop_duplicates()
        std_mae_compilation_train = pd.concat([std_mae_compilation_train, std_mae_analyze_train],
                                                ignore_index=True).drop_duplicates()
        if not SMAPE_ONLY: 
            mae_compilation_test = pd.concat([mae_compilation_test, mae_analyze_test],
                                            ignore_index=True).drop_duplicates()
            maev_compilation_test = pd.concat([maev_compilation_test, maev_analyze_test],
                                            ignore_index=True).drop_duplicates()
            mae_compilation_train = pd.concat([mae_compilation_train, mae_analyze_train],
                                            ignore_index=True).drop_duplicates()
            maev_compilation_train = pd.concat([maev_compilation_train, maev_analyze_train],
                                            ignore_index=True).drop_duplicates()
            smape_compilation_test = pd.concat([smape_compilation_test, smape_analyze_test],
                                            ignore_index=True).drop_duplicates()
            smape_compilation_train = pd.concat([smape_compilation_train, smape_analyze_train],
                                                ignore_index=True).drop_duplicates()
            
            
        

    # Sauvegarde finale
    os.makedirs(sizeDir, exist_ok=True)
    std_mae_compilation_test.to_csv(std_mae_test_file_path, index=False)
    std_mae_compilation_train.to_csv(std_mae_train_file_path, index=False)
    if not SMAPE_ONLY:
        mae_compilation_test.to_csv(mae_test_file_path, index=False)
        maev_compilation_test.to_csv(maev_test_file_path, index=False)
        mae_compilation_train.to_csv(mae_train_file_path, index=False)
        maev_compilation_train.to_csv(maev_train_file_path, index=False)
        smape_compilation_test.to_csv(smape_test_file_path, index=False)
        smape_compilation_train.to_csv(smape_train_file_path, index=False)
        

# Parallelized analysis method (excluding num_tests from parallelization)
def analyse_method(sample_sizes, disease_ratios, num_tests, robust_methods,diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION, n_jobs=-1):
    # Generate all task combinations (excluding num_tests)
    tasks = [
        (disease, sample_size, disease_ratio, num_test, harmonization_method, SYNTHETIC_SITES_VERSION, metrics, robust_methods)
        for disease in diseases
        for sample_size in sample_sizes
        for disease_ratio in disease_ratios
        for num_test in range(num_tests)
    ]

    # Run all combinations in parallel and collect file paths
    Parallel(n_jobs=n_jobs)(delayed(process_analysis)(*task) for task in tasks)  
    

## EXECUTOR

In [ ]:
harmonization_method= "gmm"

SYNTHETIC_SITES_VERSION = "v1"

metrics = get_metrics()
#diseases = get_diseases(True)
diseases = ["ALL"]
robust_methods = [
                "IQR",
                "MAD",
                "MLP3_ALL_5",
                ]

sample_sizes = [5,10,20,30,100,150]  # Différentes tailles d'échantillon
sample_sizes = [30,100,150]  # Différentes tailles d'échantillon
sample_sizes = [100]  # Différentes tailles d'échantillon
disease_ratios = [0.03, 0.1, 0.3, 0.5, 0.7, 0.8]  # Différents pourcentages de malades
num_tests = 20  # Nombre de tests à effectuer pour chaque combinaison
n_jobs_number=-1

In [ ]:
analyse_method(sample_sizes, disease_ratios, num_tests, robust_methods,diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION, n_jobs=n_jobs_number)